In [1]:
#Download required imports
import sys
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchcodec
!{sys.executable} -m pip install evaluate
!{sys.executable} -m pip install jiwer
!{sys.executable} -m pip install transformers[torch]


Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/x86-64-v3, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo2023/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo20

In [1]:
#Import dataset and split
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("rishabbahal/quebecois_canadian_french_dataset", "default", split="train")
common_voice["test"] = load_dataset("rishabbahal/quebecois_canadian_french_dataset", "default", split="test")

print(common_voice)

/home/ulaval.ca/lemun9/env_hf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 1348/1348 [00:00<00:00, 7728.31 examples/s]


DatasetDict({
    train: Dataset({
        features: ['audio', 'text', 'audio_filepath', '__index_level_0__'],
        num_rows: 5389
    })
    test: Dataset({
        features: ['audio', 'text', 'audio_filepath', '__index_level_0__'],
        num_rows: 1348
    })
})


In [ ]:
#Keep only needed data (audio and text)
common_voice = common_voice.remove_columns(["audio_filepath", "__index_level_0__"])

print(common_voice)

In [4]:
#Load feature extractor from pre-trained check-point
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [5]:
#Load tokenizer
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="French", task="transcribe")

In [6]:
#Combine feature extractor and tokenizer to create the processor
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="French", task="transcribe")

In [7]:
#Prepare data
sample = common_voice["train"][2]
audio_decoder = sample['audio']
audio_samples = audio_decoder.get_all_samples()

audio_array = audio_samples.data.squeeze()
sampling_rate = audio_samples.sample_rate

print(f"Audio array shape: {audio_array.shape}")
print(f"Sampling rate: {sampling_rate}")

Audio array shape: torch.Size([36784])
Sampling rate: 16000


In [8]:
#Prepare dataset
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]
    audio_samples = audio_decoder.get_all_samples()
    audio_array = audio_samples.data.squeeze(0).numpy()

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio_array, sampling_rate=16000).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["text"]).input_ids
    return batch

In [9]:
#Apply data preparation function to training examples
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=1)

Map (num_proc=1): 100%|██████████| 1348/1348 [00:20<00:00, 65.14 examples/s]


In [10]:
#Load pre-trained checkpoint
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

In [11]:
#Disable automatic language detection and force model to generate french
model.generation_config.language = "french"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

In [12]:
#Define data collector
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [13]:
#Initialize data collector
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [14]:
#Choose evaluation metrics (error rate)
import evaluate

metric = evaluate.load("wer")

In [15]:
#Define a function that takes the model's predictions and returns the evaluation metric
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [16]:
#Define training configuration
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir = "/scratch/lemun9@ulaval.ca/whisper_checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    eval_strategy="steps", 
    save_strategy="steps", 
    save_steps=1000,
    eval_steps=1000, 
    gradient_checkpointing=False,
    fp16=False,
    per_device_eval_batch_size=1,
    predict_with_generate=True,
    generation_max_length=225,
    logging_steps=25,
    report_to=[],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
    save_only_model=True,
    save_total_limit=2,
)

In [17]:
#Forward training agruments, model, dataset, data collector, and compute metrics function to HuggingFace trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    processing_class=processor
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [66]:
#Save processor object
processor.save_pretrained(training_args.output_dir)

[]

In [68]:
#Make space for the model training
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [69]:
#Train the model
trainer.train()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.